In [1]:
!pip install langchain huggingface_hub transformers sentence_transformers mlflow --quiet

In [2]:
import torch
import mlflow
import transformers

from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# Initialise tokenizer (Use 1xP100 GPU)

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [4]:
template = """Question Answer everything after this in English:{input}"""
prompt = PromptTemplate(template=template, input_variables=["input"])

llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )
question = "What is the capital of India?"
print(llm_chain.run(question))

sarajesh


In [5]:
template2 = """Answer the question comes after this {input}"""
prompt = PromptTemplate(template=template2, input_variables=["input"])

llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What is the capital of India?"
print(llm_chain.run(question))

sarajevo


In [6]:
template3 = """Summerize the content:  {input}"""
prompt = PromptTemplate(template=template3, input_variables=["input"])

llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

content = "The key driving force of advertising platforms is data. The more data an advertising platform has about its users, the more relevant and effective its ads can be. This data can include things like user demographics, interests, online behavior, and even purchase history."
print(llm_chain.run(content))

Search engines are the driving force of advertising platforms.


In [7]:
mlflow.langchain.log_model(lc_model=llm_chain, artifact_path="model",
                            registered_model_name="flan-t5")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 mlflow.langchain.log_model(lc_model=llm_chain, artifact_path="model",                        │
│   2 │   │   │   │   │   │   │   registered_model_name="flan-t5")                                 │
│   3                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/mlflow/langchain/__init__.py:392 in log_model            │
│                                                                                                  │
│   389 │   │   │    metadata of the logged model.                                                 │
│   390 │   """                                                                                    │
│   391 │   import langchain                                                                       │
│ ❱ 392 │   from langchain.chains import (                                                         │
│   393 │   │   RetrievalQA,                                                                       │
│   394 │   │   APIChain,                                                                          │
│   395 │   │   HypotheticalDocumentEmbedder,                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'SQLDatabaseChain' from 'langchain.chains' 
(/opt/conda/lib/python3.10/site-packages/langchain/chains/__init__.py)

In [ ]:
chat_pipeline = transformers.pipeline(model="microsoft/DialoGPT-medium")

# LLM logging: use MLflow via the Pythonic code as given below:

with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=chat_pipeline,
        artifact_path="chatbot",
        input_example="Welcome aboard!"
  )

In [ ]:
# Interactive shell style
chatbot = mlflow.pyfunc.load_model(model_info.model_uri)
chatbot.predict("Recommend a non-stop flight from London to Singapore?")

In [ ]:
chatbot.predict("Any Hindu temple to explore in Singapore?")